In [1]:
# Урок 4

# Системы управления базами данных MongoDB и SQLite в Python

In [1]:
!pip install pymongo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from pymongo import MongoClient
from pymongo.errors import  DuplicateKeyError
from pprint import pprint
from lxml import html
import requests

In [3]:
client = MongoClient('mongodb://127.0.0.1:27017/')
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
URL_YANDES_NEWS = 'https://yandex.ru/news'
URL_LENTA_NEWS = 'https://lenta.ru/'
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
db = client.testdb

In [56]:
def get_content_dom_from_html_text(url, headers=None, params=None):
    response = requests.get(url, headers=headers, params=params)
    content_dom = html.fromstring(response.text)
    
    return content_dom


def parser_yandex_news(content_dom):
    news_container = content_dom.xpath("//section//div[contains(@class, 'mg-grid__item')]")

    yandex_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//div[@class='mg-card-footer__left']//a/text()")[0]
        new_text = new_container.xpath(".//h2/a/text()")[0].replace('\xa0', ' ')
        new_url = new_container.xpath(".//h2/a/@href")[0]
        new_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")[0]
        
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_url': new_url,
            'Дата new_date': new_date,
        }
        
        yandex_news.append(news_dict)
        
    return yandex_news


def parser_lenta_news(content_dom):
    news_container = content_dom.xpath("//a[contains(@class, 'card-')]")

    lenta_news = []
    for new_container in news_container:
        new_source = new_container.xpath(".//*[name()='svg']/*[name()='use']/attribute::*")
        if len(new_source) == 1:
            new_source = new_source[0].split('ui-label_')
            if len(new_source) == 2:
                new_source = new_source[1]
            else:
                new_source = 'Lenta.ru'
        else:
            new_source = 'Lenta.ru'

            
        new_text = new_container.xpath(".//span[contains(@class, 'card-')]/text()")
        if len(new_text) == 1:
            new_text = new_text[0]
        else:
            new_text = None
            
        new_url = URL_LENTA_NEWS + new_container.xpath("./@href")[0]
        
        new_date = new_container.xpath(".//time/text()")
        if len(new_date) == 1:
            new_date = new_date[0]
        else:
            new_date = None
        
        news_dict = {
            'new_source': new_source,
            'new_text': new_text,
            'new_url': new_url,
            'Дата new_date': new_date,
        }
        
        lenta_news.append(news_dict)
        
    return lenta_news


def get_news(content_dom, parser_name='yandex'):
    result = None
    
    if parser_name == 'yandex':
        result = parser_yandex_news(content_dom)
    if parser_name == 'lenta':
        result = parser_lenta_news(content_dom)
    
    return result

def insert_new_row(parsed_file:list):
    try:
      if(type(parsed_file)==list):
        for doc in parsed_file:
          doc['_id']= doc['new_text']+doc['new_source']
          db.testdb.insert_many(doc)
      elif(type(parsed_file)==dict):
        parsed_file['_id']= parsed_file['new_text']+parsed_file['new_source']
        db.testdb.insert_one(parsed_file)
      else: 
        return("Неверный тип данных")
      return("Добавление прошло успешно")
    except:
      return("При добавлении произошла ошибка")

In [57]:
dom = get_content_dom_from_html_text(URL_YANDES_NEWS, headers=headers)
insert_new_row(get_news(dom, 'yandex'))

'Добавление прошло успешно'